# 最长回文子串
## 描述
给定一个字符串$S$，找到$S$中最长的回文子串。你可以假设$S$的最大长度为 1000。
## 解答
假设字符串为$S=c_1c_2\cdots c_{n-1}$，我们用$k(i)$表示字符串中以$c_i$为右端的最大回文串$P_i$的长度，即$P_i=c_{i-k(i)+1}\cdots c_i=c_i\cdots c_{i-k(i)+1}$，显然$1\leq k(i)\leq i+1$。

现在问题转换为求$\max\limits_{0\leq i<n}\{k(i)\}$。
具体地，当$\max\limits_{0\leq i\leq r-1}{k(i)}=k_{\max}$已知时，我们需判断$k(r)$与$k_{\max}$的关系进而考虑是否更新$k_{\max}$。
### 思路1
我们可以遍历以$c_r$为右端长度$t\in(k_{\max},r+1]$的子串是否为回文子串。

In [3]:
def longestPalindrome(s):
    if len(s) <= 1:
        return s
    k_max = [0,1]
    for r in range(len(s)):
        for t in range(r+1,k_max[1],-1):
            tmp = s[r-t+1:r+1]
            if tmp == tmp[::-1]:
                k_max = [r, t]
                break
    return s[k_max[0]-k_max[1]+1:k_max[0]+1]

在leetcode上运行时间为4896ms，战胜了28.31\%python3提交记录，内存消耗为13.7MB。
### 思路2
我们首先证明
$$k(r+1)\leq k(r)+2\tag{1}$$。
> 当$k(r+1)\leq 2$时，有$k(r+1)\leq 2\leq k(r)+2$，成立。
>
> 当$k(r+1)>2$时，则有$r-k(r+1)+2<r$，此时$P_{r+1}$可以写为如下形式
> $$
P_{r+1}=c_{r-k(r+1)+1}c_{r-k(r+1)+2}\cdots c_rc_{r+1}=c_{r+1}c_r\cdots c_{r-k(r+1)+2}c_{r-k(r+1)},
$$
> 所以
> $$
c_{r-k(r+1)+2}\cdots c_r=c_r\cdots c_{r-k(r+1)+2},
$$
> 所以
> $$
k(r)\geq r-[r-k(r+1)+2]=k(r+1)-2.
$$

基于上述结论可进一步缩小遍历范围。令$i_0=\max\{\mathop{\arg\max}\limits_{0\leq i\leq r-1}{k(i)}\}$，显然$i_0\leq r-1$，由公式(1)得$k(r)\leq k(i_0)+2(r-i_0)$，
所以我们只需搜寻是否存在长度$t\in[k_0,\min(k(i_0)+2(r-i_0),r+1)]$的回文字符串$P_r$。


In [5]:
def longestPalindrome(s):
    le = len(s)
    if le <= 1:
        return s
    k_max = [0,1]
    for r in range(le):
        for t in range(min(k_max[1]+2*(r-k_max[0]),r+1), k_max[1]-1, -1):
            tmp = s[r-t+1:r+1]
            if tmp == tmp[::-1]:
                k_max = [r, t]
                break
    return s[k_max[0]-k_max[1]+1:k_max[0]+1]

在leetcode上执行时间为2964ms，战胜52.07\%python3提交记录，内存消耗为13.5MB。
### 思路2优化
在思路2算法实现中，验证是否为回文串使用了最粗暴的方法`tmp=tmp[::-1]`。对于字符串$c_{r-t+1}\cdots c_r$，我们要验证其是否为回文串，只需依次验证$c_{r-t+1+j}=c_{r-j}$即可，其中$r-t+1+j\leq r-j$，即$0\leq j\leq\frac{t-1}{2}$，理论上当其不是回文串时等尽早地推出验证。

In [6]:
def longestPalindrome(s):
    le = len(s)
    if le <= 1:
        return s
    k_max = [0,1]
    for r in range(le):
        for t in range(min(k_max[1]+2*(r-k_max[0]),r+1), k_max[1]-1, -1):
            j = 0
            while j <= (t+1)/2 and s[r-t+1+j] == s[r-j]:
                j += 1
            if j > (t+1)/2:
                k_max = [r,t]
                break
    return s[k_max[0]-k_max[1]+1:k_max[0]+1]

相对于思路2，理论上是优化了的，但在leetcode上实际测试时，时间超出限制。。。
### 思路3
我们在思路2的基础上进一步缩小搜索范围。

对于$i_0=\max\{\mathop{\arg\max}\limits_{0\leq i\leq r-1}{k(i)}\}$，显然$(0\leq i_0\leq r-1)$。现在证明$k(r)\leq k_0+2$。
> 当$i_0<r-1$时，用反证法，假设$k(r)>k_0+2$，则有
$$k(r)>k(i_0)+2\geq 2\Rightarrow k(r)\geq 3\Rightarrow r-k(r)+1\leq r-1,$$
因此回文串$P_r$可以写为$c_{r-k(r)+1}c_{r-k(r)+2}\cdots c_{r-1}c_r$，显然$c_{r-k(r)+2}\cdots c_{r-1}$也是回文串，所以有$k(r-1)\geq k(r)-2>k(i_0)+2-2=k(i_0)=\max\limits_{0\leq i\leq r-1}k(i)$，矛盾。
> 
> 当$i_0=r-1$时，由公式(1)显然$k(r)\leq k(r-1)+2=k(i_0)+2$。

现在我们只需验证是否存在长度$t\in[k_0, \min(k_0+2,r+1)]$的回文串$P_r$。

In [2]:
def longestPalindrome(s):
    if len(s) <= 1:
        return s
    k_max = [0,1]
    for r in range(len(s)):
        for t in range(min(k_max[1]+2,r+1), k_max[1]-1, -1):
            tmp = s[r-t+1:r+1]
            if tmp == tmp[::-1]:
                k_max = [r, t]
                break
    return s[k_max[0]-k_max[1]+1:k_max[0]+1]

在leetcode上运行时间为88ms，战胜97.09\%python3提交记录，内存为13.6MB，不错。
### 思路3优化
在思路3的实现中，当$i_0=r-1$时，我们仍然使用`tmp=tmp[::-1]`判断是否回文串，事实上，此时我们最期待的情形是$k(r)=k(i_0)+2$，因为我们只需判断$c_r=c_{r-k(i_0)-1}$即可，因此在使用`tmp=tmp[::-1]`进行判断之前，我们可以先判断$c_r=c_{r-k(i_0)-1}$。

In [3]:
def longestPalindrome(s):
    if len(s) <= 1:
        return s
    k_max = [0,1]
    for r in range(1,len(s)):
        if k_max[0] == r-1 and k_max[1] <= r-1 and s[r] == s[r-k_max[1]-1]:
            k_max = [r, k_max[1]+2]
            continue
        for t in range(min(k_max[1]+2,r+1), k_max[1]-1, -1):
            tmp = s[r-t+1:r+1]
            if tmp == tmp[::-1]:
                k_max = [r, t]
                break
    return s[k_max[0]-k_max[1]+1:k_max[0]+1]

在leetcode上时间为108ms，战胜94.83\%的python3提交记录，内存为13.5MB，相对于未优化前，不升反降。。。